In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
import time

class TimeMeasurement:
    def __init__(self):
        self.lastTime = time.process_time()
        
    def getDelta(self):
        currTime = time.process_time()
        result = currTime - self.lastTime
        self.lastTime = currTime
        return result
    
tm = TimeMeasurement()

In [3]:
def det3(a, b, c):
    return (a[0]*b[1] + b[0]*c[1] + c[0]*a[1]) - (c[0]*b[1] + b[0]*a[1] + a[0]*c[1])

def orient(a, b, c, e=1e-12):
    d = det3(a, b, c)
    if d > e:
        return 1
    elif d <-e:
        return -1
    else:
        return 0
    
# def cross(a, b):
#     return a[0]*b[1] - a[1]*b[0]

class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1 #first point
        self.p2 = p2 #second point
        
    def __eq__(self, other):
        #symetric edges
        return (self.p1 == other.p1 and self.p2 == other.p2) or (self.p1 == other.p2 and self.p2 == other.p1)
    
    def __hash__(self):
        return hash((self.p1, self.p2))
    
    def __str__(self):
        return "{" + str(self.p1) + "," + str(self.p2) + "}"
    
    def __repr__(self):
        return "{" + str(self.p1) + "," + str(self.p2) + "}"
    
class Tr:
    def __init__(self, e1, e2, e3):
        #e1,e2,e3 - edges of triangle CCW
        self.edges = set([e1, e2, e3])
        self.neigh = []
        self.active = True
        
    def circumcircle(self):
        '''
        midpoint and radius of circumcircle
        https://en.wikipedia.org/wiki/Circumscribed_circle
        '''
        pts = []
        for edge in self.edges:
            pts.append(edge.p1)
        
        a = pts[0][0]**2 + pts[0][1]**2
        b = pts[1][0]**2 + pts[1][1]**2
        c = pts[2][0]**2 + pts[2][1]**2
        
        t = det3(pts[0], pts[1], pts[2])
        
        sx = (1/2) * det3([a, pts[0][1]],[b, pts[1][1]], [c, pts[2][1]]) / t
        sy = (1/2) * det3([pts[0][0], a],[pts[1][0], b], [pts[2][0], c]) / t
        s = np.array([sx, sy])
        
        return (s, np.linalg.norm(s-pts[0]))
    
    def neighBy(self, edge, triangulation):
        for n in self.neigh:
            if edge in triangulation[n].edges or Edge(edge.p2, edge.p1) in triangulation[n].edges:
                return n
    
    def contains(self, point):
        for e in self.edges:
            if det3(e.p1, e.p2, point) > 0:
                return False
        return True
    
    def circumcircleContains(self, point):
        s, r = self.circumcircle()
        return np.linalg.norm(s-point) < r
    
#     def centroid(self):
#         pts = []
#         for edge in self.edges:
#             pts.append(edge.p1)
#         s = [(pts[0][0] + pts[1][0] + pts[2][0])/3, (pts[0][1] + pts[1][1] + pts[2][1])/3]
#         return np.array(s)
    
#     def dist(self, point):
#         s = self.centroid()
#         return np.linalg.norm(s-point)
    
#     def closerNeigh(self, triangulation, pe):
#         pb = self.centroid()
#         for e in self.edges:
#             r = pe-pb
#             s = np.array(e.p2)-np.array(e.p1)
#             cr = cross(r, s)
#             if cr != 0:
#                 u = cross(np.array(e.p1)-pb, r)/cr
#                 t = cross(np.array(e.p1)-pb, s)/cr
#                 if u <= 1 and t <= 1:
#                     n = self.neighBy(e, triangulation)
#                     if n is not None:
#                         return n
    
    def __eq__(self, other):
        return len(self.edges - other.edges) == 0
    
    def __hash__(self):
        es = []
        for e in self.edges:
            es.append(e)
        return hash(tuple(es))

In [4]:
def badDFS(tr, point, visited, badTrs, triangulation):
    visited.add(tr)
    if triangulation[tr].circumcircleContains(point):
        badTrs.append(tr)
        for n in triangulation[tr].neigh:
            if n not in visited:
                badDFS(n, point, visited, badTrs, triangulation)

def triangulate(points):
    #random shuffle
    points = np.random.permutation(points)
    
    y_max, y_min = max(points[:,1].tolist()), min(points[:,1].tolist())
    x_max, x_min = max(points[:,0].tolist()), min(points[:,0].tolist())
    
    m = max([abs(y_max), abs(y_min), abs(x_max), abs(x_min)])
    
    a = []
    b = []
    
    #create super-triangle
    superTr = Tr(Edge((-10*m, -10*m), (0, 10*m)), Edge((0, 10*m), (10*m, 0)), Edge((10*m, 0), (-10*m, -10*m)))
    triangulation = [superTr]
    conTr = 0
    
    #triangulation
    #https://en.wikipedia.org/wiki/Bowyer%E2%80%93Watson_algorithm
    for p in points:
#         print("start", conTr)
#         while not triangulation[conTr].contains(p):
#             conTr = triangulation[conTr].closerNeigh(triangulation, p)
        #find triangle, which contains point
        tm.getDelta()
        for i,tr in enumerate(triangulation):
            if tr.active:
                if tr.contains(p):
                    conTr = i
                    break
        a.append(tm.getDelta())
            
        #find triangles to remove
        visited = set([conTr])
        badTrs = [conTr]
        for n in triangulation[conTr].neigh:
            badDFS(n, p, visited, badTrs, triangulation)
                
        edges = {} #number of edge occurrences in badTrs 
        for tr in badTrs:
            triangulation[tr].active = False
            for e in triangulation[tr].edges:
                if e in edges:
                    edges[e][0] += 1
                else:
                    if Edge(e.p2, e.p1) in edges: #symetric edge
                        edges[Edge(e.p2, e.p1)][0] += 1
                    else:
                        n = triangulation[tr].neighBy(e, triangulation)
                        if n is not None:
                            triangulation[n].neigh.remove(tr)
                            triangulation[tr].neigh.remove(n)
                        edges[e] = [1, n]
        
        #sort edges of polygon
        edges = [(k, v[1]) for k, v in edges.items() if v[0] == 1]
        sorted_edges = [edges[0]]
        for i in range(1, len(edges)):
            for j in range(1, len(edges)):
                if edges[j][0].p1 == sorted_edges[i-1][0].p2:
                    sorted_edges.append(edges[j])
                    break
        
        #add triangles with neighbors
        t = len(triangulation)
        for i in range(len(sorted_edges)):
            tr = Tr(sorted_edges[i][0], Edge(sorted_edges[i][0].p2, tuple(p.tolist())), Edge(tuple(p.tolist()), sorted_edges[i][0].p1))
            tr.neigh = [t+((i-1) % len(sorted_edges)), t+((i+1) % len(sorted_edges))]
            if sorted_edges[i][1] is not None:
                tr.neigh.append(sorted_edges[i][1])
                triangulation[sorted_edges[i][1]].neigh.append(t+i)
            triangulation.append(tr)
            conTr = t+i
        b.append(tm.getDelta())

    #remove edges to super-triangle
    edges = set()
    for tr in triangulation:
        if not tr.active:
            continue
        for e in tr.edges:
            toAdd = True
            for sE in superTr.edges:
                if e.p1 == sE.p1 or e.p2 == sE.p1 or e.p1 == sE.p2 or e.p2 == sE.p2:
                    toAdd = False
            if toAdd:
                edges.add(e)
    
    return sum(a), sum(b)

In [5]:
for i in range(100, 2100, 100):
    pts = np.random.uniform(-10000, 10000, size=(i, 2))
    res = triangulate(pts)
    print(res)

(0.036123141999998554, 0.06117636599999976)
(0.12276418900000152, 0.07500647099999735)
(0.2953427699999973, 0.13628542300000257)
(0.5264117730000022, 0.16535234300000035)
(0.796441339000006, 0.22059237499999673)
(1.1775515530000096, 0.23358203799998822)
(1.6320178219999963, 0.3119268440000056)
(1.9866996300000128, 0.3234814539999924)
(2.646164556000004, 0.38447166200000815)
(3.2296716679999857, 0.4017741920000386)
(3.971682681000132, 0.46297584899992117)
(5.07476148800006, 0.5506266539999167)
(6.079595615999981, 0.5744247510000804)
(6.918421265999932, 0.6144709100000867)
(8.008391925999817, 0.6569624190001235)
(9.285215887999897, 0.7335446410000301)
(10.475471751999706, 0.7308090010000683)
(12.523742546000122, 0.8527941349999963)
(13.805141205000126, 0.9286445369997125)
(15.30245242399998, 0.9841465380000471)


In [ ]:
%matplotlib notebook
def plot_res(res):
    for e in res:
        plt.plot([e.p1[0], e.p2[0]], [e.p1[1], e.p2[1]])

In [ ]:
pts = np.random.uniform(-10000, 10000, size=(1000, 2))
res = triangulate(pts)
plot_res(res)